In [ ]:
from google.colab import auth
from google.cloud import bigquery
import pandas as pd
import os
from google.colab import files

projectID = 'finalproject-ecmms'
location = 'US'
client = bigquery.Client(project=projectID, location=location)

auth.authenticate_user()

print('authenticated')

In [ ]:
queries = {
    '3-products-profit-bw': """
        WITH products_profit AS (
          SELECT
            p.id AS product_id,
            name AS product_name,
            category,
            ROUND(retail_price) AS retail_price,
            ROUND(cost) AS cost,
            ROUND(SUM(sale_price - cost)) AS profit
          FROM `bigquery-public-data.thelook_ecommerce.products` AS p
          JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS o
          ON p.id = o.product_id
          WHERE status NOT IN ('Returned', 'Canceled')
          GROUP BY product_id, product_name, category, retail_price, cost
        )
        SELECT *
        FROM (
          SELECT *
          FROM products_profit
          ORDER BY profit DESC
          LIMIT 5
        )
        UNION ALL
        (
          SELECT *
          FROM products_profit
          ORDER BY profit ASC
          LIMIT 5
        )
    """,
    '3-brands-profit-bw': """
        WITH brand_sales_revenue AS (
          SELECT
            p.brand,
            COUNT(*) AS quantity,
            ROUND(SUM(oi.sale_price - p.cost)) AS revenue
          FROM `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
            ON oi.product_id = p.id
          WHERE oi.status NOT IN ('Returned', 'Canceled')
          GROUP BY p.brand
        )
        SELECT *
        FROM (
          SELECT brand, quantity, revenue
          FROM brand_sales_revenue
          ORDER BY revenue DESC
          LIMIT 15
        )
        UNION ALL
        (
          SELECT brand, quantity, revenue
          FROM brand_sales_revenue
          ORDER BY quantity DESC
          LIMIT 15
        )
    """,
    '3-product-sales-profit': """
        SELECT
          p.id AS product_id,
          name AS product_name,
          category,
          ROUND(retail_price) AS retail_price,
          ROUND(cost) AS cost,
          COUNT(*) AS purchase_count,
          ROUND(SUM(sale_price - cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.products` AS p
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS o
        ON p.id = o.product_id
        WHERE status NOT IN ('Returned', 'Canceled')
        GROUP BY
          product_id,
          product_name,
          category,
          retail_price,
          cost
        ORDER BY purchase_count DESC
    """,
    '3-categories-age': """
        SELECT p.category,
          COUNT(CASE WHEN u.age < 18 THEN 1 END) AS under_18,
          COUNT(CASE WHEN u.age BETWEEN 18 AND 30 THEN 1 END) AS between_18_30,
          COUNT(CASE WHEN u.age BETWEEN 30 AND 50 THEN 1 END) AS between_30_50,
          COUNT(CASE WHEN u.age > 50 THEN 1 END) AS over_50
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        GROUP BY p.category
    """,
    '3-categories-gender': """
        SELECT p.category,
          COUNT(CASE WHEN u.gender = 'M' THEN 1 END) AS M,
          COUNT(CASE WHEN u.gender = 'F' THEN 1 END) AS F
        FROM `bigquery-public-data.thelook_ecommerce.orders` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.order_items` AS oi
          ON oi.order_id = o.order_id
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
          ON oi.product_id = p.id
        JOIN `bigquery-public-data.thelook_ecommerce.users` AS u
          ON o.user_id = u.id
        GROUP BY p.category
    """,
    '3-categories-profit': """
        SELECT
          p.category, COUNT(*) AS quantity,
          ROUND(SUM(o.sale_price - p.cost)) AS profit
        FROM `bigquery-public-data.thelook_ecommerce.order_items` AS o
        JOIN `bigquery-public-data.thelook_ecommerce.products` AS p
        ON p.id = o.product_id
        WHERE status NOT IN ('Returned', 'Canceled')
        GROUP BY category
        ORDER BY profit DESC
    """
}

for query_name, query in queries.items():
    query_job = client.query(query)
    results = query_job.result().to_dataframe()
    filename = f'{query_name}.csv'
    results.to_csv(filename, index=False)
    print(f'Risultati "{query_name}" salvati come CSV')
    files.download(filename)

print('Download completato')
